In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
# URM_train, URM_val = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")

In [6]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [7]:
from libs.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from libs.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

In [8]:
def objective_function_P3_beta(optuna_trial):
       
        
    recommender_instance = RP3betaRecommender(URM_train)
    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 2, 20),
                             alpha = optuna_trial.suggest_float("alpha",0.2, 0.5),
                             beta = optuna_trial.suggest_float("beta",0.1, 0.5),
                             normalize_similarity= True,
                             implicit=False,
                            )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)    
    return eval_res["MAP"][10]

In [9]:
def objective_function_P3_alpha(optuna_trial):

    
                          
        
    recommender_instance = P3alphaRecommender(URM_train)
    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 2, 400),
                             alpha=  optuna_trial.suggest_float("alpha",0.35, 0.39,log=False),
                             normalize_similarity= True
                            )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)    
    return eval_res["MAP"][10]

In [10]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        
        self.results_df = self.results_df._append(hyperparam_dict, ignore_index=True)

In [11]:
optuna_study_beta = op.create_study(direction="maximize")
        
save_results_beta = SaveResults()
        
optuna_study_beta.optimize(objective_function_P3_beta,
                      callbacks=[save_results_beta],
                      n_trials = 300)

[I 2024-12-17 23:11:49,927] A new study created in memory with name: no-name-8a93c79d-fb11-422d-9a3b-e71261ab5e14


RP3betaRecommender: Similarity column 38121 (100.0%), 4617.94 column/sec. Elapsed time 8.25 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.28 sec. Users per second: 4195


[I 2024-12-17 23:12:06,806] Trial 0 finished with value: 0.027526442457117252 and parameters: {'topK': 9, 'alpha': 0.22946274021782684, 'beta': 0.33512048110906417}. Best is trial 0 with value: 0.027526442457117252.


RP3betaRecommender: Similarity column 38121 (100.0%), 4617.69 column/sec. Elapsed time 8.26 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.26 sec. Users per second: 4207


[I 2024-12-17 23:12:23,584] Trial 1 finished with value: 0.025936235067627066 and parameters: {'topK': 5, 'alpha': 0.25413451124563885, 'beta': 0.22340562916485285}. Best is trial 0 with value: 0.027526442457117252.


...

[I 2024-12-18 00:45:25,946] Trial 297 finished with value: 0.02493693735076717 and parameters: {'topK': 5, 'alpha': 0.361661317137725, 'beta': 0.4871338876956504}. Best is trial 73 with value: 0.028440063002086992.


RP3betaRecommender: Similarity column 38121 (100.0%), 4151.79 column/sec. Elapsed time 9.18 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 10.13 sec. Users per second: 3430


[I 2024-12-18 00:45:45,738] Trial 298 finished with value: 0.028174178102477326 and parameters: {'topK': 17, 'alpha': 0.3839816662656345, 'beta': 0.29066924105926206}. Best is trial 73 with value: 0.028440063002086992.


RP3betaRecommender: Similarity column 38121 (100.0%), 4126.03 column/sec. Elapsed time 9.24 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 9.63 sec. Users per second: 3608


[I 2024-12-18 00:46:05,059] Trial 299 finished with value: 0.028426159029993795 and parameters: {'topK': 17, 'alpha': 0.3768048491882805, 'beta': 0.21359264866197847}. Best is trial 73 with value: 0.028440063002086992.


In [12]:
save_results_beta.results_df.to_csv("result_experiments/RP3beta/results_optuna_20.csv")

In [15]:
optuna_study_alpha = op.create_study(direction="maximize")
        
save_results_alpha = SaveResults()
        
optuna_study_alpha.optimize(objective_function_P3_alpha,
                      callbacks=[save_results_alpha],
                      n_trials = 300)

[I 2024-12-09 07:40:21,634] A new study created in memory with name: no-name-190db610-7b6e-4d85-a877-2611764cbb27


P3alphaRecommender: Similarity column 38121 (100.0%), 4703.27 column/sec. Elapsed time 8.11 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 9.02 sec. Users per second: 3946


[I 2024-12-09 07:40:39,405] Trial 0 finished with value: 0.05047467220517467 and parameters: {'topK': 51, 'alpha': 0.3774241480510347}. Best is trial 0 with value: 0.05047467220517467.


P3alphaRecommender: Similarity column 38121 (100.0%), 4690.72 column/sec. Elapsed time 8.13 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 9.03 sec. Users per second: 3941


[I 2024-12-09 07:40:57,216] Trial 1 finished with value: 0.05040746556503141 and parameters: {'topK': 52, 'alpha': 0.3846647742101469}. Best is trial 0 with value: 0.05047467220517467.


...

[I 2024-12-09 09:11:02,485] Trial 297 finished with value: 0.05193420476065364 and parameters: {'topK': 35, 'alpha': 0.35649162064736145}. Best is trial 101 with value: 0.054204765117559894.


P3alphaRecommender: Similarity column 38121 (100.0%), 5014.79 column/sec. Elapsed time 7.60 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 8.50 sec. Users per second: 4187


[I 2024-12-09 09:11:18,922] Trial 298 finished with value: 0.05394699259627957 and parameters: {'topK': 16, 'alpha': 0.3549334113308688}. Best is trial 101 with value: 0.054204765117559894.


P3alphaRecommender: Similarity column 38121 (100.0%), 5082.31 column/sec. Elapsed time 7.50 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 8.31 sec. Users per second: 4279


[I 2024-12-09 09:11:35,008] Trial 299 finished with value: 0.05328089553224295 and parameters: {'topK': 8, 'alpha': 0.3511059201165217}. Best is trial 101 with value: 0.054204765117559894.


In [17]:
save_results_alpha.results_df.to_csv("result_experiments/RP3alpha/results_optuna.csv")

In [5]:
results_alpha = pd.read_csv("result_experiments/RP3alpha/results_optuna.csv")
results_beta = pd.read_csv("result_experiments/RP3beta/results_optuna_50.csv")

In [7]:
results_beta.sort_values(by=['result'], ascending=False)

,Unnamed: 0,topK,alpha,beta,result
235,235,12.0,0.349899,0.181734,0.055589
267,267,12.0,0.358275,0.183071,0.055585
232,232,12.0,0.352013,0.170365,0.055554
249,249,12.0,0.318763,0.178043,0.055539
265,265,12.0,0.359616,0.178041,0.055538
...,...,...,...,...,...
195,195,2.0,0.863639,0.172310,0.035370
7,7,29.0,0.579395,0.922559,0.034820
133,133,14.0,0.282290,0.985331,0.034568
9,9,48.0,0.293029,0.919261,0.033317


In [8]:
results_alpha.sort_values(by=['result'], ascending=False)

,Unnamed: 0,topK,alpha,result
101,101,13.0,0.358969,0.054205
264,264,13.0,0.356510,0.054203
280,280,13.0,0.359350,0.054201
97,97,13.0,0.356190,0.054192
267,267,13.0,0.355121,0.054187
...,...,...,...,...
154,154,2.0,0.350523,0.036901
213,213,2.0,0.350160,0.036896
220,220,2.0,0.350028,0.036895
248,248,2.0,0.350073,0.036895
